# Analisis de datos


In [38]:
import pandas as pd

data = pd.read_csv("datos_analisis.csv")


### Sobre el número total de noticias:
**¿Cuál es el número total de noticias recopiladas?**

In [ ]:
num_filas = data.shape[0]
print(f"El numero total de noticias es {num_filas}")

El numero total de noticias es 719


### Número de noticias por mes:
- **¿Cuántas noticias se publicaron cada mes?**
- **¿Cuál fue el mes con más y menos noticias publicadas?**

### Frecuencia de longitud del contenido:
- **¿Cuál es la distribución de las longitudes de las noticias en palabras?**
- **¿Cuál es la longitud promedio de las noticias?**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df_palabras = pd.DataFrame(data)

def contar_palabras(fila):
    return len(fila["Contenido"].split())

df_palabras["Total_Palabras"] = df_palabras.apply(contar_palabras, axis = 1)


#print(frecuencias)


     Total_Palabras  Frecuencia
0                 5           1
1                33           2
2                34           1
3                36           1
4                37           1
..              ...         ...
363             890           1
364             909           1
365             934           1
366            1001           1
367            1104           1

[368 rows x 2 columns]


### Palabras comunes:
**¿Cuáles son las palabras más frecuentes en las noticias?**

### Frecuencia de la ubicación de las noticias:
- **¿Cuáles son las ubicaciones más mencionadas en las noticias?**
- **¿Existen patrones geográficos claros en las noticias publicadas?**

### Análisis del traductor maya-español:
- **¿Cuántas palabras en las noticias están escritas en maya?**
- **¿Cuántas palabras provienen del español?**
- **¿Existen palabras que aparecen tanto en maya como en español?**

In [29]:
from collections import Counter
import re

# Función para generar n-gramas
def generate_ngrams(text, n=3):
    text = text.lower()
    text = ''.join([c for c in text if c.isalpha() or c.isspace()])
    ngrams = [text[i:i+n] for i in range(len(text)-n+1)]
    return Counter(ngrams)

# Función para cargar el contenido del archivo
def load_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Cargar los textos en Maya Yucateco y Español desde archivos de texto
maya_text = load_text_from_file('maya_text.txt')  # Ruta al archivo de Maya Yucateco
spanish_text = load_text_from_file('spanish.txt')  # Ruta al archivo de Español

# Crear los perfiles de n-gramas para Maya Yucateco y Español
language_profiles = {
    'maya': generate_ngrams(maya_text, n=3),
    'es': generate_ngrams(spanish_text, n=3)
}

# Función para calcular la proporción
def proportion_similarity(profile1, profile2):
    intersection = set(profile1.keys()) & set(profile2.keys())
    matches = sum(profile1[ngram] for ngram in intersection)
    total = sum(profile1.values())
    return matches / total if total > 0 else 0.0

# Función para detectar el idioma
def detect_language(text):
    text_profile = generate_ngrams(text, n=3)
    similarities = {lang: proportion_similarity(text_profile, profile) for lang, profile in language_profiles.items()}
    detected_language = max(similarities, key=similarities.get)
    return detected_language, similarities

# Ejemplo de uso
text_to_detect = data["Contenido"][200]
detected_language, similarities = detect_language(text_to_detect)

print(f"Idioma detectado: {detected_language}")
print(f"Similitudes: {similarities}")

Idioma detectado: maya
Similitudes: {'maya': 0.876699484294421, 'es': 0.5410220346929208}


In [36]:
# Cargar el diccionario de español
def load_spanish_dictionary(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        # Leer todas las palabras en el diccionario y convertirlas a una lista
        spanish_dict = set(word.strip().lower() for word in file.readlines())
    return spanish_dict

# Función para contar palabras en maya y español
def count_words(text, spanish_dict):
    maya_count = 0
    spanish_count = 0
    words = text.split()  # Divide el texto en palabras
    
    for word in words:
        word_cleaned = word.strip('.,;!?()[]{}"').lower()  # Elimina signos de puntuación
        if word_cleaned in spanish_dict:
            spanish_count += 1
        else:
            maya_count += 1  # Si no está en el diccionario de español, asumimos que es maya
    
    return maya_count, spanish_count

# Cargar el diccionario de español desde un archivo
spanish_dict = load_spanish_dictionary('0_palabras_todas.txt')

# Ejemplo de texto para analizar
text = data["Contenido"][10]
print(text)

# Llamar la función
maya_count, spanish_count = count_words(text, spanish_dict)

print(f"Palabras en maya: {maya_count}")
print(f"Palabras en español: {spanish_count}")


Kaanpech yéetel Yucatáne’ náach p’aatalo’ob ti’ Quintana Roo ikil u táakbesa’alo’ob ti’ fíinansiero ba’alo’ob. Le ts’ook lu’uma’ ti’ yaan tu ka’ kúuchil uchil tuláakal u noj lu’umil México, tumen jo’op’éel u méek’tankaajilo’obe’ ka’anal yanik u káalifikaasiono’obi’, le beetike’ ti’ kaaje’ ku yantal uláak’ páajtalilo’ob ti’al u páajtal u machiko’ob taak’in yéetel beyo’ u ma’alo’obkúunsa’al u kuxtal máaxo’ob óoli’ p’ata’ano’ob paachil, je’el bix ko’olelo’ob, táaxnel kaajil máako’ob, ch’ija’an máako’ob, máasewalo’ob yéetel máaxo’ob kaja’ano’ob ti’ mejen kaajo’ob. 
Kaanpeche’ jatsa’an beey nivel medio tumen ti’ yaan tu jo’olajun kúuchil ti’ ba’ax k’ajóolta’an beey Índice de Inclusión Financiera 2020, beeta’an tumen Citibanamex, ka’alikil Yucatáne’ ti’ yaan tu waklajun kúuchil. 
Xookil ku beeta’ale’ ku núup’ul yéetel Política Nacional de Inclusión Financiera jts’a’ab k’ajóoltbil le ja’ab máaniko’ tu winalil marzo, tu’ux táan u kaxta’al u yantal uláak’ ba’alo’ob ti’al u yutsil kuxtal kaaj, u

In [44]:
maya_count = 0
spanish_count = 0
for index, row in data.iterrows():
    texto = row['Contenido']
    mc, sc = count_words(texto, spanish_dict)
    maya_count += mc
    spanish_count += sc
print(f"se encontraron un total de {spanish_count} palabras en español")
print(f"se encontraron un total de {maya_count} palabras en maya")

se encontraron un total de 51372 palabras en español
se encontraron un total de 157662 palabras en maya
